# Introduction

[pywhip](https://inbo.github.io/pywhip/) provides the ability to **validate a dataset** and receive a **report to identify potential issues** using [whip specifications](https://github.com/inbo/whip), a human and machine-readable syntax to express specifications for data.

In this notebook, we introduce the pywhip functionalities. As an introduction, we present a dummy example of the workflow to apply pywhip. Next, the usage in the context of a real-world example from the [TrIAS project](https://github.com/trias-project/) is shown. Finally, some envisioned ideas about pywhip applications are shortly discussed.

If you want to run the code in the code blocks (called "cells"), select the cell and do `Shift + Enter` (or use `Cell > Run Cells` from the menu at the top).

In [ ]:
import yaml
from pywhip import whip_csv

## pywhip workflow with dummy data

Assume we received the following dataset from a research project that took place in Belgium and The Netherlands between 2016 and 2018:

In [ ]:
!head ../docs/_static/observations_data.csv

Within the scope of the project, we do know the following about the data:

- The project was happening in Belgium (BE) and The Netherlands (NL)
- The project was running from 2016 until 2018, so date values should be in this range
- Individual counts can not be higher than 100 and should be at least 1
- Empty values are not allowed

These specifications can be translated to [whip specifications](https://github.com/inbo/whip):

In [ ]:
project_specs = """
    country:
       allowed: [BE, NL]
    eventDate:
        dateformat: '%Y-%m-%d'
        mindate: 2016-01-01
        maxdate: 2018-12-31
    individualCount:
        numberformat: x  # needs to be an integer value
        min: 1
        max: 100
"""
specifications = yaml.load(project_specs)

**pywhip** provides the ability to **test** these specifications:

In [ ]:
import yaml
observations_whip = whip_csv("../docs/_static/observations_data.csv", specifications, delimiter=',')

and **report** the issues to the user:

In [ ]:
html_report = observations_whip.get_report('html')

Using the Jupyter Notebook build-in functionalities, we can show the HTML-report here inline, but this page could be served elsewhere as well:

In [ ]:
from IPython.display import HTML, display_html
display_html(HTML(html_report), 
             metadata=dict(isolated=True))

The report can also be served in the json format, allowing integration with other websites and services:

In [ ]:
observations_whip.get_report('json')

## Real-world case

The [Tracking Invasive Alien Species (TrIAS)](http://trias-project.be) project aims to build an open data-driven framework to support policy on invasive species. One of its aims is to standardize and publish occurrence and checklist data so these can be harvested by [GBIF](http://www.gbif.org/).

The [GBIF data validator](https://www.gbif.org/tools/data-validator) allows to validate TrIAS data for and within the context of GBIF. However, those tests are predefined. Using whip + pywhip, the publisher can **define, document and test their own rules** (whether these are generic or specific), complementing what the GBIF data validator offers.

As an example, the [alien-macroinvertebrates repository](https://github.com/trias-project/alien-macroinvertebrates) contains the functionality to standardize the data of Boets et al. (2016) to both a Darwin Core checklist and Darwin Core occurrence dataset. For each of the data sets, whip specifications where defined and added to the repository, see [this link](https://github.com/trias-project/alien-macroinvertebrates/tree/master/specification).

We can use these specifications to validate the status of each of the datasets. To illustrate this, we'll use the (small) `taxon.csv` dataset:

Reading the specifications from the URL:

In [ ]:
import requests
alien_macroinvertebrates_yaml = 'https://raw.githubusercontent.com/trias-project/alien-macroinvertebrates/ccd9025adfed3ee6a710da73213d439f5ac89506/specification/dwc_taxon.yaml'
response = requests.get(alien_macroinvertebrates_yaml)
alien_macroinvertebrates_specifications = yaml.load(response.text)

A user working on the data, can check the data against the specifications:

In [ ]:
checklist_whip = whip_csv("alien-taxon.csv", 
                          alien_macroinvertebrates_specifications, delimiter=',')

And check the report:

In [ ]:
display_html(HTML(checklist_whip.get_report('html')), 
             metadata=dict(isolated=True))

The report provides a quick overview of the issues for which action needs to be taken, similar to the current GBIF data validator reporting functionalities.

## pywhip envisioned applications

In essence, pywhip is a data validation tool, but one where you can define your own rules (in [whip](https://github.com/inbo/whip)). We envision the usefulness of whip + pywhip in a number of applications:

- Data publishers can use whip to **document decisions** made about data representation/standardization (e.g. in discussions with the data owner). These specifications could even be included in the published dataset (e.g. in a Darwin Core Archive). Use pywhip they can test these specifications in an iternative process to improve the standardization and quality of the published dataset.
- Data users can use whip to express their custom or **community-defined** data quality needs. Using pywhip, data meets those requirements can be _filtered_ and extracted.
- Pywhip functionalities could be **integrated** into the [GBIF data validator](https://www.gbif.org/tools/data-validator), so that in addition to the predefined rules, user-defined rules could tested and reported upon as well.
- The TDWG [Biodiversity Data Quality (BDQ) Interest Group](https://github.com/tdwg/bdq) is defining a fixed set of _Tests and Assertions_ to assess data quality and provide a common ground for data aggregators to report these issues. Whip could be considered to define existing or additional conformance tests.